In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Example using a solution with constant rhs

Let $c_1=-\frac{e^2 + e}{e^2 + e + 1}$ $c_1=-\frac{1}{e^2 + e + 1}$, and $u(x)=c_1e^{x} + c_2 e^{-2x} + 1$. Then, using $\alpha=1$, $b=1$ and $c=2$, we have $f(x)=-(Lu)(x) = 1$.

In [2]:
print(1+1)

2
